In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [4]:
import pandas as pd
from numpy import mean, isnan
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [5]:
def get_training_dataset(binary=False):
    df = pd.read_csv('/content/drive/MyDrive/DSTTProject/data/UNSW_NB15_training-set_cleaned.csv')
    labels = df.iloc[:, -1] if binary else df.iloc[:, -2]
    return df.iloc[:, 0:-2], labels

def get_testing_dataset(binary=False):
    df = pd.read_csv('/content/drive/MyDrive/DSTTProject/data/UNSW_NB15_testing-set_cleaned.csv')
    labels = df.iloc[:, -1] if binary else df.iloc[:, -2]
    return df.iloc[:, 0:-2], labels

In [6]:
def get_models():
    models = (LinearSVC(), SVC())
    for model in models:
      yield model

def test_bin_model(model, scaler):
    X_train, y_train = get_training_dataset(binary=True)
    X_test, y_test = get_testing_dataset(binary=True)

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    model.fit(X_train, y_train)
    y_preds = model.predict(X_test)

    return accuracy_score(y_test, y_preds), f1_score(y_test, y_preds)

def test_mult_model(model, scaler=None):
    X_train, y_train = get_training_dataset()
    X_test, y_test = get_testing_dataset()

    if scaler:
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)

    model.fit(X_train, y_train)
    ovr_clf = OneVsRestClassifier(model, n_jobs=-1).fit(X_train, y_train)
    ovo_clf = OneVsOneClassifier(model, n_jobs=-1).fit(X_train, y_train)

    y_preds = model.predict(X_test)
    ovr_y_preds = ovr_clf.predict(X_test)
    ovo_y_preds = ovo_clf.predict(X_test)

    return (accuracy_score(y_test, y_preds), f1_score(y_test, y_preds, average='weighted'),
    accuracy_score(y_test, ovr_y_preds), f1_score(y_test, ovr_y_preds, average='weighted'),
    accuracy_score(y_test, ovo_y_preds), f1_score(y_test, ovo_y_preds, average='weighted'))

In [7]:
def print_results(scaler, accuracy, f1, ovr_accuracy, ovr_f1, ovo_accuracy, ovo_f1):
    print("")

    print(f"\t{scaler}:")

    print("")

    print(f"\t\tSimple:")
    if not isnan(accuracy):
      print(f"\t\t\taccuracy: {accuracy}")
    if not isnan(f1):
      print(f"\t\t\tf1-score: {f1}")

    print("")

    print(f"\t\tOne vs Rest:")
    if not isnan(ovr_accuracy):
      print(f"\t\t\taccuracy: {ovr_accuracy}")
    if not isnan(ovr_f1):
      print(f"\t\t\tf1-score: {ovr_f1}")

    print("")

    print(f"\t\tOne vs One:")
    if not isnan(ovo_accuracy):
      print(f"\t\t\taccuracy: {ovo_accuracy}")
    if not isnan(ovo_f1):
      print(f"\t\t\tf1-score: {ovo_f1}")

    print("")

def eval(model):
    print(f"{type(model).__name__}:")
    print_results("Without scaler", *test_mult_model(model))
    print_results("Standard scaler", *test_mult_model(model, StandardScaler()))
    print_results("Min-Max scaler", *test_mult_model(model, MinMaxScaler()))


In [11]:
model = get_models()

In [ ]:
eval(next(model))

LinearSVC:

	Without scaler:

		Simple:
			accuracy: 0.11961834368459173
			f1-score: 0.15042308661300124

		One vs Rest:
			accuracy: 0.3522393507508227
			f1-score: 0.3346352237000804

		One vs One:
			accuracy: 0.3537678010277117
			f1-score: 0.26609373369411493


	Standard scaler:

		Simple:
			accuracy: 0.6943954922123177
			f1-score: 0.6404819913560311

		One vs Rest:
			accuracy: 0.6939677542616958
			f1-score: 0.6384155871687213

		One vs One:
			accuracy: 0.713455495292031
			f1-score: 0.669204469619744


	Min-Max scaler:

		Simple:
			accuracy: 0.6882075498599871
			f1-score: 0.6326035794137278

		One vs Rest:
			accuracy: 0.6882075498599871
			f1-score: 0.6326035794137278

		One vs One:
			accuracy: 0.7055394916191877
			f1-score: 0.6583161853358944



In [ ]:
eval(next(model))

SVC:
